# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [ ]:
pip install geoalchemy2 geopandas shapely psycopg2 psycopg2-binary sqlalchemy matplotlib requests

In [ ]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse
import os
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db
import psycopg2


from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base

In [ ]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path(".")
ZIPCODE_DATA_FILE = pathlib.Path("data")/ "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "8ITaLVGKJEzelLCfrNyuIi2rJ"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "project_database"
DB_USER = "mac"
DB_URL = f"postgresql://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [ ]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():

    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [ ]:
def download_nyc_geojson_data(url, jsonname,force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / (url_path.split('/')[-1])
    
    if force or not filename.exists():
        print(f"Downloading {url} to {jsonname}...")
        response = requests.get(url)
        response.raise_for_status()
       
        with open(jsonname, "a") as f:
            json.dump(response.json(), f)
        print(f"Done downloading {url}.")
    else:
        print(f"Reading from {jsonname}...")

    return jsonname

In [ ]:
def clean_311_data(df):
    columns = ['unique_key', 'created_date', 'incident_zip', 'complaint_type','longitude','latitude']
    df_selected=df[columns]
    df_selected=df_selected.dropna(axis=0)
    
    df_selected.rename(columns={'incident_zip': 'zipcode'}, inplace=True)
    df_selected['zipcode']=df_selected['zipcode'].astype(int)
    df_selected.rename(columns={'unique_key': 'id'}, inplace=True)
    df_selected.rename(columns={'created_date': 'date'}, inplace=True)
    df_selected['geometry']=gpd.points_from_xy(df_selected['longitude'],df_selected['latitude'],crs="EPSG:4326")
    df_selected=df_selected.drop(['latitude','longitude'],axis=1)
    df_selected['date'] = pd.to_datetime(df_selected['date']).dt.strftime('%Y-%m-%d')
    
    return gpd.GeoDataFrame(df_selected,geometry='geometry')

In [ ]:
def download_311_data(offset):
    base_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_311}?$$app_token={NYC_DATA_APP_TOKEN}"
    url = f'{base_url}&$limit=1000000&$offset={offset}&$where=created_date between"2018-01-01T12:00:00.000"and"2023-09-30T12:00:00.000"'
    download_nyc_geojson_data(url,jsonname=f'query_{offset/1000000+1}_data.geojson',force=False)
    print('begin to create csv')
    df=gpd.read_file(f'query_{offset/1000000+1}_data.geojson')
    print('finishing create csv')
    df.to_csv(f'query_{offset/1000000+1}_data.csv', index=False)
    return clean_311_data(pd.read_csv(f'query_{offset/1000000+1}_data.csv'))

### zipcode data

In [ ]:
def load_and_clean_zipcodes(zipcode_datafile):
    gdf = gpd.read_file(zipcode_datafile)
    #normalized data
    gdf_normalized = gdf.to_crs(epsg=4326)
    
    #selected_zipcode
    columns=['ZIPCODE', 'PO_NAME', 'STATE','COUNTY','geometry']
    gdf_selected=gdf_normalized[columns]
    gdf_selected.rename(columns={'ZIPCODE': 'zipcode'}, inplace=True)
    gdf_selected.rename(columns={'COUNTY': 'county'}, inplace=True)
    gdf_selected.rename(columns={'STATE': 'state'}, inplace=True)
    gdf_selected.rename(columns={'PO_NAME': 'poname'}, inplace=True)
    #cleaned_zipcode
    gdf_cleaned = gdf_selected.dropna()
    return gdf_cleaned

### tree data

In [9]:
def load_and_clean_tree_data():
    base_url = f"{BASE_NYC_DATA_URL}{NYC_DATA_TREES}?$$app_token={NYC_DATA_APP_TOKEN}"
    url = f'{base_url}&$limit=1000000'
    download_nyc_geojson_data(url,jsonname='nyc_tree.geojson',force=False)
    df = gpd.read_file('nyc_tree.geojson')
    df.to_csv('nyc_tree.csv',index=False)
    
    df_read=pd.read_csv("nyc_tree.csv")
    columns = ['tree_id','zipcode','longitude', 'latitude','spc_common','health','status']
    df_selected = df_read[columns]
    df_selected.rename(columns={'tree_id': 'id'}, inplace=True)
    df_selected['geometry']=gpd.points_from_xy(df_selected['longitude'],df_selected['latitude'],crs="EPSG:4326")
    
    df_drop=df_selected.drop(['latitude','longitude'],axis=1)
    df_cleaned = df_drop.dropna()
    return gpd.GeoDataFrame(df_cleaned,geometry='geometry')

### zillow data

In [1]:
def load_and_clean_zillow_data(ZILLOW_DATA_FILE):
    df = pd.read_csv(ZILLOW_DATA_FILE)
    # only choose NY data
    filtered_df = df[df['State'] == 'NY']
    #change data from wide to long
    df_melt=filtered_df.melt(id_vars=['RegionName'],value_vars=['2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31',
       '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31','2015-11-30',
       '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30',
       '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30',
       '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28',
       '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31',
       '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31',
       '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31',
       '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31',
       '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31',
       '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31',
       '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31',
       '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30',
       '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
       '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
       '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30',
       '2021-10-31', '2021-11-30', '2021-12-31', '2022-01-31', '2022-02-28',
       '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31',
       '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31',
       '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31',
       '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30'],var_name='date',value_name='rent')
    df_cleaned = df_melt.dropna()
    df_cleaned.rename(columns={'RegionName': 'zipcode'}, inplace=True)
    return df_cleaned

### load all data

#### nyc_311 data has download limitation, so we split it into 25 files

In [ ]:
df1=download_311_data(1*1000000)
df2=download_311_data(2*1000000)
df3=download_311_data(3*1000000)
df4=download_311_data(4*1000000)
df5=download_311_data(5*1000000)
df6=download_311_data(6*1000000)
df7=download_311_data(7*1000000)
df8=download_311_data(8*1000000)
df9=download_311_data(9*1000000)
df10=download_311_data(10*1000000)
df11=download_311_data(11*1000000)
df12=download_311_data(12*1000000)
df13=download_311_data(13*1000000)
df14=download_311_data(14*1000000)
df15=download_311_data(15*1000000)
df16=download_311_data(16*1000000)
df17=download_311_data(17*1000000)
df18=download_311_data(18*1000000)
df19=download_311_data(19*1000000)
df20=download_311_data(20*1000000)
df21=download_311_data(21*1000000)
df22=download_311_data(22*1000000)
df23=download_311_data(23*1000000)
df24=download_311_data(24*1000000)
df25=download_311_data(25*1000000)
df26=download_311_data(26*1000000)
df27=download_311_data(27*1000000)

In [ ]:
import dask.dataframe as dd
df1=pd.read_csv(pathlib.Path("data-311")/'query_1_data.csv')
df2=pd.read_csv(pathlib.Path("data-311")/'query_2_data.csv')
df3=pd.read_csv(pathlib.Path("data-311")/'query_3_data.csv')
df4=pd.read_csv(pathlib.Path("data-311")/'query_4_data.csv')
df5=pd.read_csv(pathlib.Path("data-311")/'query_5_data.csv')
df6=pd.read_csv(pathlib.Path("data-311")/'query_6_data.csv')
df7=pd.read_csv(pathlib.Path("data-311")/'query_7_data.csv')
df8=pd.read_csv(pathlib.Path("data-311")/'query_8_data.csv')
df9=pd.read_csv(pathlib.Path("data-311")/'query_9_data.csv')
df10=pd.read_csv(pathlib.Path("data-311")/'query_10_data.csv')
df11=pd.read_csv(pathlib.Path("data-311")/'query_11_data.csv')
df12=pd.read_csv(pathlib.Path("data-311")/'query_12_data.csv')
df13=pd.read_csv(pathlib.Path("data-311")/'query_13_data.csv')
df14=pd.read_csv(pathlib.Path("data-311")/'query_14_data.csv')
df15=pd.read_csv(pathlib.Path("data-311")/'query_15_data.csv')
df16=pd.read_csv(pathlib.Path("data-311")/'query_16_data.csv')
df17=pd.read_csv(pathlib.Path("data-311")/'query_17_data.csv')
df18=pd.read_csv(pathlib.Path("data-311")/'query_18_data.csv')
df19=pd.read_csv(pathlib.Path("data-311")/'query_19_data.csv')
df20=pd.read_csv(pathlib.Path("data-311")/'query_20_data.csv')
df21=pd.read_csv(pathlib.Path("data-311")/'query_21_data.csv')
df22=pd.read_csv(pathlib.Path("data-311")/'query_22_data.csv')
df23=pd.read_csv(pathlib.Path("data-311")/'query_23_data.csv')
df24=pd.read_csv(pathlib.Path("data-311")/'query_24_data.csv')
df25=pd.read_csv(pathlib.Path("data-311")/'query_25_data.csv')

In [ ]:
#concat the data into one dataframe
df_311=pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25], ignore_index=True)

In [8]:
def load_all_data(df):
    geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
    geodf_311_data = clean_311_data(df)
    geodf_tree_data = load_and_clean_tree_data()
    geodf_zillow_data = load_and_clean_zillow_data(ZILLOW_DATA_FILE)
    return (
        geodf_zipcode_data,
        geodf_311_data,
        geodf_tree_data,
        geodf_zillow_data
    )

In [ ]:
geodf_zipcode_data, geodf_311_data, geodf_tree_data, geodf_zillow_data = load_all_data(df_311)

### show dataframe 

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
geodf_zillow_data.info()

In [ ]:
geodf_zillow_data.head(10)

## Part 2: Storing Data

### Creating databese

In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
connection = psycopg2.connect(dbname="postgres",user=DB_USER,host="localhost",port="5432" )
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = connection.cursor()
cursor.execute("CREATE DATABASE {}".format(DB_NAME))
print(f"create {DB_NAME} sucessfully!")

### Creating extension

In [ ]:
connection1 = psycopg2.connect(dbname=DB_NAME,user=DB_USER,host="localhost",port="5432" )
cursor1 = connection1.cursor()
cursor1.execute('CREATE EXTENSION postgis;')

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_zipcode(
    zipcode INTEGER ,
    poname VARCHAR,
    state VARCHAR,
    county VARCHAR,
    geometry geometry(Polygon, 4326)
);

"""

NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_311(
    id SERIAL PRIMARY KEY,
    date DATE,
    zipcode INTEGER,
    complaint_type VARCHAR,
    geometry GEOMETRY(Point, 4326)
);
"""

NYC_TREE_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_tree(
    id SERIAL PRIMARY KEY,
    zipcode INTEGER,
    spc_common VARCHAR,
    health VARCHAR,
    status VARCHAR,
    geometry GEOMETRY(Point, 4326)
);
"""

ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS zillow(
    zipcode INTEGER,
    date DATE,
    rent DECIMAL
);
"""

indance='''
CREATE INDEX idx_geometries_geom_gist ON zipcode USING gist (geometry)
'''

In [ ]:
# create that required schema.sql file
with open("schema.sql", "w") as f:
    f.write(ZIPCODE_SCHEMA)
   
    f.write(NYC_311_SCHEMA)
  
    f.write(NYC_TREE_SCHEMA)

    f.write(ZILLOW_SCHEMA)
    
    f.write(indance)

In [ ]:
with open("schema.sql", 'r') as file:
    sql_commands = file.read()
commands = sql_commands.split(';')

for command in commands:
    if command.strip(): 
        print(command)
        connection2 = psycopg2.connect(dbname=DB_NAME,user=DB_USER,host="localhost",port="5432" )
        cursor2 = connection2.cursor()
        cursor2.execute(command)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

In [ ]:
def write_dataframes_to_table(DB_USER,DB_NAME):
    engine = create_engine(f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}")
    geodf_zipcode_data.to_postgis('zipcode', engine, if_exists='append', index=False)
    geodf_311_data.to_postgis('nyc_311', engine, if_exists='append', index=False)
    geodf_tree_data.to_postgis('nyc_tree', engine, if_exists='append', index=False)
    geodf_zillow_data.to_sql('zillow', engine, if_exists='append', index=False)
    
write_dataframes_to_table(DB_USER,DB_NAME)

## Part 3: Understanding the Data

### Query 1

Query 1: Which area might be more calm to live in?

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(QUERY_1, QUERY_1_FILENAME):
    with open (QUERY_1_FILENAME,'w')as f:
        f.write(QUERY_1)
        f.close

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "query1"

QUERY_1 = """


"""
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

### Query 2

Query 2: Where has the most greenery?
Zipcode
tree_id
Using just the trees table, which 10 zip codes have the most trees?

The query result should have two columns, 10 rows. The rows should be sorted by the total number of trees, descending.


In [ ]:
QUERY_2_FILENAME = QUERY_DIR / "query2"

QUERY_2 = """

"""
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_2))
    for row in result:
        print(row)

### Query 3


Query 3: Can I afford a place in the areas with the most trees?
Zillow_rent
RegionName, 2023-08-31


Of the 10 zip codes with the most trees, for the month of August 2023, what is the average rent by zip code?

The query should have a JOIN statement. The query result should have two columns (not three) and 10 rows. The rows should be sorted by the total number of trees, descending. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128. 



In [ ]:
QUERY_3_FILENAME = QUERY_DIR / "query3"

QUERY_3 = """

"""
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_3))
    for row in result:
        print(row)

### Query4
Query 4: Could there be a correlation between an area’s rent, the number of its trees, and the number of 311 complaints?
2023-1
For the month of January 2023, return the 5 zip codes with the lowest average rent, and 5 zipcodes of the highest average rent, and include the tree count and complaint count for each zip code by using JOIN statements.

The query result should have 4 columns (zip code, average rent, tree count, and complaint count) and 10 rows: five with the highest average rent, and five with the lowest average rent. “Humanize” the rent numbers, meaning format the results as 2,879.58 instead of 2879.575128.


In [ ]:
QUERY_4_FILENAME = QUERY_DIR / "query4"

QUERY_4 = """

"""
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_4))
    for row in result:
        print(row)

### Query 5

Query 5: Where has the most greenery (take 2)?
Rewrite Query 2 to use both the trees table and the zipcodes table. Join both tables where the coordinate point of the tree is inside the polygon boundary of the zipcode as defined in the zipcode table.



The query should have a JOIN statement. The query results should match exactly the results of Query 2.



In [ ]:
QUERY_5_FILENAME = QUERY_DIR / "query5"

QUERY_5 = """


"""
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_5))
    for row in result:
        print(row)

### Query 6

Using the following coordinate pair on campus, which trees are within ½ mile radius of this
point?
Latitude: 40.80737875669467, Longitude: -73.96253174434912
Tree: longitude, latitude, tree_id, spc_common, health, status, x_sp, y_sp
The result should have 5 columns (ID, species, health, status, and coordinate location of each
tree).

In [ ]:
QUERY_6_FILENAME = QUERY_DIR / "query6"

QUERY_6 = """

"""
write_query_to_file(QUERY_6, QUERY_6_FILENAME)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_6))
    for row in result:
        print(row)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)